# Работа с веб-камерой:

1. Базовое отображение видео потока

In [16]:
import cv2
import numpy as np

# Инициализация веб-камеры
cap = cv2.VideoCapture(1)

# Проверка, успешно ли открыта веб-камера
if not cap.isOpened():
    print("Не удалось открыть веб-камеру")

# Установка разрешения на 720p (1280x720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    # Захват кадра с веб-камеры
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр с веб-камеры")
        break

    # Отображение кадра
    cv2.imshow('Webcam', frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

2. Обработка кадра

In [17]:
def convert_to_grayscale(frame):
    """
    Преобразует кадр в оттенки серого.
    
    :param frame: Исходный кадр
    :return: Кадр в оттенках серого
    """
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return gray_frame

Вычисление FPS:

In [3]:
import time

class FPS_Counter:
    def __init__(self, calc_time_perion_N_frames: int) -> None:
        """Счетчик FPS по ограниченным участкам видео (скользящему окну).

        Args:
            calc_time_perion_N_frames (int): количество фреймов окна подсчета статистики.
        """
        self.time_buffer = []
        self.calc_time_perion_N_frames = calc_time_perion_N_frames

    def calc_FPS(self) -> float:
        """Производит рассчет FPS по нескольким кадрам видео.

        Returns:
            float: значение FPS.
        """
        time_buffer_is_full = len(self.time_buffer) == self.calc_time_perion_N_frames
        t = time.time()
        self.time_buffer.append(t)

        if time_buffer_is_full:
            self.time_buffer.pop(0)
            fps = len(self.time_buffer) / (self.time_buffer[-1] - self.time_buffer[0])
            return np.round(fps, 2)
        else:
            return 0.0


def calc_and_show_fps(frame, fps_counter):
    """
    Вычисляет и отображает FPS на кадре.

    Args:
        frame (numpy.ndarray): Текущий кадр изображения.
        fps_counter (FPS_Counter): Объект класса FPS_Counter для вычисления FPS.

    Returns:
        numpy.ndarray: Кадр с отображенным значением FPS.
    """
    fps_real = fps_counter.calc_FPS()
    text = f"FPS: {fps_real:.1f}"

    # Параметры для шрифтов:
    fontFace = 1
    fontScale = 2.3
    thickness = 1
    
    (label_width, label_height), _ = cv2.getTextSize(
        text,
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness,
    )
    frame = cv2.rectangle(frame, (0, 0), (10 + label_width, 15 + label_height), (0, 0, 0), -1)
    frame = cv2.putText(
        img=frame,
        text=text,
        org=(5, 20),
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness,
        color=(255, 255, 255),
    )
    return frame

In [20]:
import cv2
import time

# Инициализация веб-камеры
cap = cv2.VideoCapture(1)

counter = FPS_Counter(calc_time_perion_N_frames=5)

# Проверка, успешно ли открыта веб-камера
if not cap.isOpened():
    print("Не удалось открыть веб-камеру")

# Установка разрешения на 720p (1280x720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    # Захват кадра с веб-камеры
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр с веб-камеры")
        break

    frame = convert_to_grayscale(frame)
    time.sleep(0.1)
    frame = calc_and_show_fps(frame, counter)

    # ресайз
    scale = 0.5
    frame = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)

    # Отображение кадра
    cv2.imshow('Webcam', frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

---

In [23]:
def display_green_objects(frame):
    """
    Выводит исходный кадр и под ним бинарную маску, выделяющую зеленые объекты.
    
    :param frame: Исходный кадр
    :return: Кадр исходный + бинарная маска зеленых пикселей снизу
    """
    # Преобразование кадра в HSV
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Определение диапазона зеленого цвета в HSV
    lower_green = np.array([60, 155, 50])
    upper_green = np.array([97, 255, 255])
    
    # Создание маски для зеленых объектов
    green_mask = cv2.inRange(hsv_frame, lower_green, upper_green)

    # Применение морфологических операций для удаления артефактов
    kernel = np.ones((15, 15), np.uint8)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)
    
    # Объединение исходного кадра и маски
    combined_frame = np.vstack((frame, np.repeat(green_mask[:, :, np.newaxis], 3, axis=2)))
    return combined_frame

In [24]:
fps_counter = FPS_Counter(calc_time_perion_N_frames=10) # Определяем размер усреднения

# Инициализация веб-камеры
cap = cv2.VideoCapture(1)

# Проверка, успешно ли открыта веб-камера
if not cap.isOpened():
    print("Не удалось открыть веб-камеру")

# Установка разрешения на 720p (1280x720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    # Захват кадра с веб-камеры
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр с веб-камеры")
        break

    #frame = convert_to_grayscale(frame)
    frame = display_green_objects(frame)

    # ресайз
    scale = 0.5
    frame = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)

    frame = calc_and_show_fps(frame, fps_counter)

    # Отображение кадра
    cv2.imshow('Webcam', frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

Изменим фон:

In [25]:
def display_green_objects(frame, background_image):
    """
    Выводит исходный кадр с заменой фона на background_image.
    
    :param frame: Исходный кадр
    :param background_image: Фоновое изображение
    :return: Кадр с замененным фоном
    """
    # Преобразование кадра в HSV
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Определение диапазона зеленого цвета в HSV
    lower_green = np.array([60, 155, 50])
    upper_green = np.array([97, 255, 255])
    
    # Создание маски для зеленых объектов
    green_mask = cv2.inRange(hsv_frame, lower_green, upper_green)

    # Применение морфологических операций для удаления артефактов
    kernel = np.ones((15, 15), np.uint8)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)
    
    # Инвертирование маски
    green_mask_inv = cv2.bitwise_not(green_mask)

    # Ресайз фонового изображения до размеров кадра
    background_image_resized = cv2.resize(background_image, (frame.shape[1], frame.shape[0]))

    # Замена фона
    background = cv2.bitwise_and(background_image_resized, background_image_resized, mask=green_mask)
    foreground = cv2.bitwise_and(frame, frame, mask=green_mask_inv)
    result = cv2.add(background, foreground)

    return result

# Загрузка фонового изображения
background_image = cv2.imread('data/otus.png')
if background_image is None:
    print("Не удалось загрузить фоновое изображение")

# Инициализация веб-камеры
cap = cv2.VideoCapture(1)

# Проверка, успешно ли открыта веб-камера
if not cap.isOpened():
    print("Не удалось открыть веб-камеру")

# Установка разрешения на 720p (1280x720)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    # Захват кадра с веб-камеры
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр с веб-камеры")
        break

    # Замена фона
    frame = display_green_objects(frame, background_image)

    # Ресайз
    scale = 0.5
    frame = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)

    # Отображение кадра
    cv2.imshow('Webcam', frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()